# Lab6.2: Topic modeling using gensim

In this notebook, we demonstrate how LDA models can be built and applied using the *gensim* package.

Credits:

This notebook is an adaptation of a blog from Susan Li's:

https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24



The data set we’ll use is a list of over one million news headlines published over a period of 15 years and can be downloaded from:

https://www.kaggle.com/therohk/million-headlines/data

We read the CSV file using the pandas framework.

In [45]:
import pandas as pd

#### Adapt the path below to point to your local copy of the data set
data = pd.read_csv('C:/Users/zakar/Downloads/abcnews-date-text.csv/abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

C:\Users\zakar\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


Let's have a look at the data:

In [46]:
print(len(documents))
print(documents[:5])

1226258
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


We are going to use the *gensim* package to build our LDA models from the data.
Before building the model, we are going to preprocess the texts.

## Data Pre-processing
We will perform the following steps:

* Tokenization: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All stopwords are removed.
* Words are lemmatized — words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are stemmed — words are reduced to their root form.

In order to apply these processing steps, we first load the gensim and nltk libraries

In [47]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zakar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
def lemmatize_stemming(text):
    return lemmatizer.lemmatize(text)
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
           # result.append(token)
            result.append(lemmatize_stemming(token))
    return result

In [49]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepayer', 'group', 'want', 'compulsory', 'local', 'govt', 'voting']


We now apply the preprocessing to all the headlines and print the first 10 results

In [50]:
processed_docs = documents['headline_text'].map(preprocess)
### print the first 10 results
processed_docs[:10]

0          [decides, community, broadcasting, licence]
1                         [witness, aware, defamation]
2           [call, infrastructure, protection, summit]
3                          [staff, aust, strike, rise]
4              [strike, affect, australian, traveller]
5               [ambitious, olsson, win, triple, jump]
6          [antic, delighted, record, breaking, barca]
7    [aussie, qualifier, stosur, waste, memphis, ma...
8             [aust, address, security, council, iraq]
9                       [australia, locked, timetable]
Name: headline_text, dtype: object

## Bag of Words on the Data set
Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.
We are going to use the *Dictionary* function to derive a dictionary with counts from the headlines.

In [51]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcasting
1 community
2 decides
3 licence
4 aware
5 defamation
6 witness
7 call
8 infrastructure
9 protection
10 summit


## Gensim filter_extremes
Filter out tokens that appear in
less than 15 documents (absolute number) or
more than 0.5 documents (fraction of total corpus size, not absolute number).
after the above two steps, keep only the first 100000 most frequent tokens.

In [52]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

## Gensim doc2bow
For each document we create a dictionary reporting how many words and how many times those words appear. 
Gensim provides the *doc2bow* function to create a BoW vector representation for a document.
Save this to ‘bow_corpus’, then check our selected document earlier.

In [53]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(164, 1), (241, 1), (615, 1), (891, 1), (4170, 1), (4171, 1), (4172, 1)]

Preview Bag Of Words for our sample preprocessed document.

In [11]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 164 ("govt") appears 1 time.
Word 241 ("group") appears 1 time.
Word 615 ("local") appears 1 time.
Word 891 ("want") appears 1 time.
Word 4170 ("compulsory") appears 1 time.
Word 4171 ("ratepayer") appears 1 time.
Word 4172 ("voting") appears 1 time.


## TF-IDF
Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.


In [13]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.6151016896202017),
 (1, 0.33155913313072555),
 (2, 0.5687633538435918),
 (3, 0.4338510112798681)]


## Running LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’. This takes a while.
Look at the documentation of *gensim* for further details:

https://radimrehurek.com/gensim/models/ldamulticore.html

As parameters, we pass the corpus data as BoW (a list of lists of tuples), the prefixed number of topics, the actual words and the number of passes and workers used for modeling.

In [17]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=3, id2word=dictionary, passes=2, workers=2)

For each topic, we will explore the words occuring in that topic and its relative weight.

In [18]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.021*"australia" + 0.014*"australian" + 0.013*"trump" + 0.013*"coronavirus" + 0.011*"queensland" + 0.007*"world" + 0.007*"south" + 0.007*"home" + 0.006*"sydney" + 0.006*"coast"
Topic: 1 
Words: 0.016*"coronavirus" + 0.009*"victoria" + 0.009*"say" + 0.009*"government" + 0.008*"covid" + 0.007*"election" + 0.006*"news" + 0.006*"health" + 0.006*"donald" + 0.006*"market"
Topic: 2 
Words: 0.018*"police" + 0.012*"woman" + 0.010*"death" + 0.010*"case" + 0.009*"court" + 0.008*"china" + 0.007*"child" + 0.007*"murder" + 0.007*"year" + 0.006*"family"


Can you distinguish different topics using the words in each topic and their corresponding weights?

## Running LDA using TF-IDF

In [19]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=3, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.008*"coronavirus" + 0.007*"australia" + 0.005*"queensland" + 0.004*"coast" + 0.004*"covid" + 0.004*"live" + 0.004*"south" + 0.004*"farmer" + 0.004*"victoria" + 0.003*"record"
Topic: 1 Word: 0.009*"police" + 0.007*"woman" + 0.006*"news" + 0.006*"death" + 0.005*"market" + 0.005*"crash" + 0.004*"australian" + 0.004*"murder" + 0.004*"charged" + 0.004*"drum"
Topic: 2 Word: 0.008*"trump" + 0.006*"coronavirus" + 0.005*"election" + 0.005*"government" + 0.005*"donald" + 0.005*"rural" + 0.004*"say" + 0.004*"country" + 0.004*"health" + 0.004*"change"


Again, can you distinguish different topics using the words in each topic and their corresponding weights? Do you observe any differences with the BoW version? Do these differences make sense given the information value weighing by the *tfidf* method?

## Performance evaluation by classifying sample document using LDA Bag of Words model
We will check where our test document would be classified.

In [20]:
processed_docs[4310]

['ratepayer', 'group', 'want', 'compulsory', 'local', 'govt', 'voting']

Document 4310 is already represented in the correct way. We can directly pass it to our *lda_model* to get the similarity scores for each topic. We represent each topic by printing 

In [21]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.7918939590454102	 
Topic: 0.016*"coronavirus" + 0.009*"victoria" + 0.009*"say" + 0.009*"government" + 0.008*"covid" + 0.007*"election" + 0.006*"news" + 0.006*"health" + 0.006*"donald" + 0.006*"market"

Score: 0.16640225052833557	 
Topic: 0.018*"police" + 0.012*"woman" + 0.010*"death" + 0.010*"case" + 0.009*"court" + 0.008*"china" + 0.007*"child" + 0.007*"murder" + 0.007*"year" + 0.006*"family"

Score: 0.04170382767915726	 
Topic: 0.021*"australia" + 0.014*"australian" + 0.013*"trump" + 0.013*"coronavirus" + 0.011*"queensland" + 0.007*"world" + 0.007*"south" + 0.007*"home" + 0.006*"sydney" + 0.006*"coast"


Our test document has the highest probability to be part of the topic that our model assigned, which is the accurate classification.

### Analyzing our LDA model

Now that we have a trained model let’s visualize the topics for interpretability. 
To do so, we’ll use a popular visualization package, *pyLDAvis* which is designed to help interactively with:

1. Better understanding and interpreting individual topics, and
2. Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most frequent and/or “relevant” terms, using different values of the λ parameter. This can help when you’re trying to assign a human interpretable name or “meaning” to each topic.
For (2), exploring the Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

You need to install *pyldavis* through the command line, following the instructions:

https://anaconda.org/conda-forge/pyldavis

WARNING: running the next cell takes a long time and you need some memory to run it. However, the result is spectacular.

In [38]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim_models
vis = pyLDAvis.gensim_models.prepare(topic_model=lda_model, corpus=bow_corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

C:\Users\zakar\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


## Some other useful functions

In [23]:
#get the top 20 words and their weights for a specific topic
topic_id=1
top_terms=20
for wordid, score in lda_model.get_topic_terms(topic_id, top_terms):
    print(wordid, ":", dictionary[wordid], ":", score)

18728 : coronavirus : 0.016075281
1619 : victoria : 0.00904366
413 : say : 0.008934315
2722 : government : 0.008834556
18729 : covid : 0.008303832
1019 : election : 0.0067601716
1363 : news : 0.006373339
259 : health : 0.0060235555
1685 : donald : 0.005822168
1188 : market : 0.0057088602
177 : change : 0.0054224967
59 : state : 0.005312021
64 : report : 0.0046559083
190 : national : 0.0046167485
363 : restriction : 0.0044603865
47 : plan : 0.004272207
7 : call : 0.0042111743
369 : hospital : 0.0041790423
81 : business : 0.0041181184
466 : federal : 0.00407966


In [30]:
#### Utility function to get the id for a word

def get_id_for_word(dictionary, word):
    for k, v in dictionary.iteritems():
        if (v==word):
            return k
    return -1

In [31]:
top_terms=20
index=get_id_for_word(dictionary,'market')
for topic_id, score in lda_model.get_term_topics(index):
    print("Topic:", topic_id)
    for wordid, score in lda_model.get_topic_terms(topic_id, top_terms):
        print(wordid, ":", dictionary[wordid], ":", score)


## Saving and loading your model for re-use

Building a model takes time.Once you have a stable model, you can save it to disk and reload it later.

In [32]:
# Save model to disk.
temp_file = "./model"
lda_model.save(temp_file)

# Load a potentially pretrained model from disk.
loaded_lda = LdaModel.load(temp_file)

NameError: name 'LdaModel' is not defined

## Testing model on unseen document

In [25]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'

In order to compare any new text against the topic model, we first need to process it in the same way as we processed the input texts for the model.
We apply the same preprocessing function and next apply the *doc2bow* function to represent it using the same vector representation as we used for modeling.

In [26]:
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
print(bow_vector)


[(888, 1), (1088, 1), (1920, 1), (5640, 1), (12290, 1)]


We can now pass this representation of the unseen document into the model to compare it against all the topics.
The next function returns in index to the topics and a similarity score for the new document. We print the scores and the topics with the top 5 words.

In [29]:
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic_id {}\t Topic: {}".format(score, index, lda_model.print_topic(index, 5)))

Score: 0.3501623868942261	 Topic_id 5	 Topic: 0.028*"australian" + 0.026*"election" + 0.019*"china" + 0.019*"donald" + 0.019*"south"
Score: 0.35012122988700867	 Topic_id 1	 Topic: 0.047*"trump" + 0.024*"market" + 0.024*"home" + 0.018*"sydney" + 0.016*"island"
Score: 0.18297365307807922	 Topic_id 2	 Topic: 0.027*"world" + 0.017*"australian" + 0.016*"tasmania" + 0.015*"farmer" + 0.014*"final"
Score: 0.01668226346373558	 Topic_id 0	 Topic: 0.018*"open" + 0.015*"tasmanian" + 0.015*"case" + 0.013*"health" + 0.011*"liberal"
Score: 0.016678284853696823	 Topic_id 7	 Topic: 0.018*"government" + 0.015*"change" + 0.014*"federal" + 0.013*"rural" + 0.013*"country"
Score: 0.016676433384418488	 Topic_id 3	 Topic: 0.022*"crash" + 0.022*"adelaide" + 0.016*"dy" + 0.016*"people" + 0.016*"royal"
Score: 0.016676433384418488	 Topic_id 4	 Topic: 0.018*"test" + 0.013*"plan" + 0.012*"australia" + 0.012*"work" + 0.011*"park"
Score: 0.016676433384418488	 Topic_id 6	 Topic: 0.021*"water" + 0.019*"help" + 0.016*"v

This text matches best with topic 5 although the score is not very high!

### Updating the model with a new document

We can also use the unseen documents to extend our model and update the topics. This is useful when processing texts in a stream.

In [30]:
# Update the model by incrementally training on the new corpus.

other_texts = [['computer', 'time', 'graph'],['survey', 'response', 'eps'],['human', 'system', 'computer']]
other_corpus = [dictionary.doc2bow(text) for text in other_texts]

# Update the model by incrementally training on the new corpus.
lda_model.update(other_corpus)  # update the LDA model with additional documents



/Users/piek/opt/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:824: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words


## End of this notebook